In [71]:
import pandas as pd
import re
from fill_variables import *
from patterns import *
from docx import Document
import PyPDF2
import mysql.connector

In [72]:
#Global file ------------------------------------------------------------------------------------------------------------
template = pd.read_excel("template.xlsx")
grades_Folder = "grades/"
subject_Folder = "subject/"
students_answer_Folder = "students_answer/"
file = get_file(grades_Folder)
subject = get_file(subject_Folder)
# -----------------------------------------------------------------------------------------------------------------------

In [73]:
config = {
    'user': 'goodgrades',
    'password': 'goodgrades',
    'host': '127.0.0.1',
    'database': 'goodgrades',
}

In [74]:
cnx = mysql.connector.connect(**config)
cur = cnx.cursor(buffered=True)

In [75]:
cur.execute("""CREATE TABLE IF NOT EXISTS goodgrades_2 (
id VARCHAR(30),
utilisateur VARCHAR(70),
ecole VARCHAR(30),
langue VARCHAR(30),
annees VARCHAR(20),
session VARCHAR(50),
competence VARCHAR(50),
diplome VARCHAR(70),
question_type INT,
question_numero INT,
support VARCHAR(2000),
enonce VARCHAR(1000),
reponse_referente VARCHAR(1000),
reponse_apprenant VARCHAR(1000),
note_sur_10 FLOAT,
note FLOAT,
note_sur FLOAT)"""
)

In [76]:
#Get question grade ------------------------------------------------------------------------------------------------------
def get_question_grade(file, grade_pattern):
    target_columns = []
    for column in file.columns.tolist():
        if re.search(grade_pattern, str(file.at[0, column])):
            target_columns.append(column)
    return target_columns
# ------------------------------------------------------------------------------------------------------------------------

In [77]:
#Extract bareme ---------------------------------------------------------------------------------------------------------
def extract_bareme(column):
    max_Grade_pattern = r"Max grade:\s*(\d+)"
    if re.search(max_Grade_pattern, column):
        match = re.search(max_Grade_pattern, column)
        return float(match.group(1))
        #insert_data_sql = f"INSERT INTO goodgrades (note_sur) VALUES ({value})"
        #value = float(match.group(1))
        #cur.execute(insert_data_sql, value)
        #template.at[j, "note_sur"] = float(match.group(1))
# ------------------------------------------------------------------------------------------------------------------------

In [18]:
grade = pd.read_excel(grades_Folder + file)

In [20]:
extract_bareme(grade, r"Grade:\s*(\d+\.?\d*)", get_question_grade(grade, r"Grade:\s*(\d+\.?\d*)")[0])

2.0

In [16]:
pattern

['Q1\xa0: ANALYSE DES RISQUES',
 'Q2\xa0: DECLENCHEMENT DE L’AUDIT',
 'Q3\xa0: PLANIFICATION DE L’AUDIT',
 'Q4\xa0: ANALYSE DOCUMENTAIRE',
 'Q5\xa0: REALISATION DE L’AUDIT',
 'Q6\xa0: CONCLUSIONS DE L’AUDIT',
 'Q7\xa0: RECHERCHE DES CAUSES',
 'Q8\xa0: SOLUTIONS']

In [78]:
#Extract questions -------------------------------------------------------------------------------------------------------
def extract_between(file, start_pattern, end_pattern):
    extracted_paragraphs = ""
    if file.endswith(".docx"):
        file = Document(file)
        is_inside_paragraph = False
        for paragraph in file.paragraphs:
            if re.search(end_pattern, paragraph.text, re.I):
                break
            if re.search(start_pattern, paragraph.text, re.I):
                is_inside_paragraph = True
            if is_inside_paragraph:
                extracted_paragraphs += ("\n" + paragraph.text)
    # elif file.endswith(".pdf"):
    #     file = PyPDF2.PdfReader(file)
    #     for page_num in range(file.numPages):
    #         page = file.getPage(page_num)
    #         text += page.extract_text()
        

    return extracted_paragraphs
# ------------------------------------------------------------------------------------------------------------------------

In [79]:
def insert_data(table_name, data_dict):
    # Generate the SQL statement dynamically
    columns = ', '.join(data_dict.keys())
    placeholders = ', '.join(['%s' for _ in data_dict.values()])
    insert_data_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    
    # Extract the values from the data_dict
    values = tuple(data_dict.values())
    
    # Execute the SQL statement with the values
    cur.execute(insert_data_sql, values)

In [80]:
#Extract name and grades -------------------------------------------------------------------------------------------------
def extract_all(file, bareme, name_index, column, grade_pattern):
    if re.search(grade_pattern, str(file.at[name_index, column])):
        match = re.search(grade_pattern, str(file.at[name_index, column]))
        data = {
            "note" : float(match.group(1)),
            "note_sur" : bareme,
            "utilisateur" : file.iloc[name_index, 0],
            "note_sur_10" : round(float(match.group(1))*10/bareme, 2)
        }
        insert_data("goodgrades_2", data)
        # template.at[j, "note"] = float(match.group(1))
        # template.at[j, "note_sur"] = bareme
        # template.at[j, "utilisateur"] = file.iloc[name_index, 0]
        #->template.at[j, "note_sur_10"] = round(template.at[j, "note"]*10/template.at[j, "note_sur"], 2)
    else :
        data = {
            "note" : float(file.at[name_index, column]),
            "note_sur" : bareme,
            "utilisateur" : file.iloc[name_index, 0],
            "note_sur_10" : round(float(match.group(1))*10/bareme, 2)
        }
        insert_data("goodgrades_2", data)
        # template.at[j, "note"] = float(file.at[name_index, column])
        # template.at[j, "note_sur"] = bareme
        # template.at[j, "utilisateur"] = file.iloc[name_index, 0]
        #->template.at[j, "note_sur_10"] = round(template.at[j, "note"]*10/template.at[j, "note_sur"], 2)
#-------------------------------------------------------------------------------------------------------------------------

In [81]:
#Extraction --------------------------------------------------------------------------------------------------------------
def run_extraction(grade):
    file = pd.read_excel(grade)
    grade_pattern = r"Grade:\s*(\d+\.?\d*)"
    target_columns = get_question_grade(file, grade_pattern) #Take the list of cells who contains the grades 
    start_pattern, end_pattern = get_patterns(subject_Folder + subject) #Take the patterns
    j,k = 0,0 #For looping the row and filling baremes
    for name_index in range(file.shape[0]):
        for column in target_columns:
            extract_all(file, extract_bareme(column), name_index, column, grade_pattern) #Extract name and grades - bareme -> template.at[k, "note_sur"]
            
            # template.at[j, "enonce"] = extract_between(subject_Folder + subject, start_pattern[k], end_pattern[k]) #Extract questions
            # answer_Folder = students_answer_Folder + file.iloc[name_index][0] + "/"
            # template.at[j, "reponse_apprenant"] = extract_between(answer_Folder + get_file(answer_Folder), start_pattern[k], end_pattern[k])
            # if k>len(target_columns)-2:
            #     k = 0
            # else :
            #     k += 1
            # j += 1
    #template.to_excel("tmplt.xlsx")
# ------------------------------------------------------------------------------------------------------------------------

In [85]:
extraction(grades_Folder + file)

In [86]:
select_all_rows_sql = "SELECT * FROM goodgrades_2"

# Execute the SQL statement to retrieve all rows from the table
cur.execute(select_all_rows_sql)

# Fetch all the rows from the result set
rows = cur.fetchall()

# Print the contents of the table
if rows:
    print("Table contents:")
    for row in rows:
        print(row)
else:
    print("Table is empty")

Table is empty
